In [8]:
import csv
import datetime
import ipynbname
import math
import numpy
import os
import pandas
import pathlib
import skimage.io
import sklearn.model_selection
import tensorflow
import time

from metrics import dice_coef, jaccard_distance
from model import unet_model, get_loss_function

In [9]:
cfg = {
    "channel": 3,
    "batch_size": 4,
    "fold": 5,
    "epochs": 75,
    "image_size": 512,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_dataset": "dataset",
    "path_out": "out",
    "loss_function": "dice",
    "data_augmentation": False
}
images_folder = os.path.join(cfg["path_dataset"], "IMAGEM_ORIGINAL", "CONVERTIDAS", "RGB", "512", "OUT")
masks_folder = os.path.join(cfg["path_dataset"], "MASK", "BITMAP", "512", "OUT")

In [10]:
list_images = sorted(list([file for file in pathlib.Path(images_folder).rglob("*")]))
list_masks = sorted(list([file for file in pathlib.Path(masks_folder).rglob("*")]))

x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(list_images, list_masks, test_size=cfg["test_size"], random_state=cfg["random_state"])
len(x_train), len(x_val)

(300, 75)

In [11]:
class CreateSequence(tensorflow.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = numpy.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = skimage.io.imread(path)
            img = numpy.float32(img/255)
            x[j] = img
        y = numpy.zeros((self.batch_size,) + self.img_size, dtype="float32")
        for j, path in enumerate(batch_target_img_paths):
            img = skimage.io.imread(path)
            img = numpy.float32(img/255)
            y[j] = img
        return x, y

In [12]:
train_data = CreateSequence(cfg["batch_size"], (cfg["image_size"], cfg["image_size"]), list_images, list_masks)
val_data = CreateSequence(cfg["batch_size"], (cfg["image_size"], cfg["image_size"]), x_val, y_val)

In [13]:
current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
path_model = os.path.join(cfg["path_out"], "train", current_datetime)
pathlib.Path(path_model).mkdir(parents=True, exist_ok=True)

steps_per_epoch = math.ceil(len(images_folder) / cfg["batch_size"])
reduce_learning_rate = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=1)
filename_model = os.path.join(path_model, "unet.h5")
checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
strategy = tensorflow.distribute.MirroredStrategy()

with strategy.scope():
    model = unet_model(cfg)
    adam_opt = tensorflow.keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
    model.compile(optimizer=adam_opt, loss=get_loss_function(cfg["loss_function"]), metrics=[dice_coef, jaccard_distance, tensorflow.keras.metrics.Precision(), tensorflow.keras.metrics.Recall()])

tensorflow.keras.backend.clear_session()
start_time = time.time()
fit = model.fit(train_data, steps_per_epoch=steps_per_epoch, epochs=cfg["epochs"], validation_data=val_data, callbacks=[checkpointer, reduce_learning_rate])
end_time = time.time() - start_time

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Epoch 1/75


2022-09-12 19:15:45.636781: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_83038"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\023FlatMapDataset:1525"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size:

12/12 [==============================] - ETA: 0s - loss: 0.6058 - dice_coef: 0.3942 - jaccard_distance: 0.2551 - precision: 0.3768 - recall: 0.9559

2022-09-12 19:15:52.140092: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_88499"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\023FlatMapDataset:1547"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size:


Epoch 1: val_loss improved from inf to 0.71720, saving model to out/train/12-09-2022-19-15-44/unet.h5
12/12 [==============================] - 9s 418ms/step - loss: 0.6058 - dice_coef: 0.3942 - jaccard_distance: 0.2551 - precision: 0.3768 - recall: 0.9559 - val_loss: 0.7172 - val_dice_coef: 0.2828 - val_jaccard_distance: 0.1658 - val_precision: 0.4646 - val_recall: 0.1234 - lr: 0.0010
Epoch 2/75
12/12 [==============================] - ETA: 0s - loss: 0.5109 - dice_coef: 0.4891 - jaccard_distance: 0.3340 - precision: 0.5642 - recall: 0.9685
Epoch 2: val_loss improved from 0.71720 to 0.71691, saving model to out/train/12-09-2022-19-15-44/unet.h5
12/12 [==============================] - 3s 280ms/step - loss: 0.5109 - dice_coef: 0.4891 - jaccard_distance: 0.3340 - precision: 0.5642 - recall: 0.9685 - val_loss: 0.7169 - val_dice_coef: 0.2831 - val_jaccard_distance: 0.1665 - val_precision: 0.2916 - val_recall: 0.0557 - lr: 0.0010
Epoch 3/75
12/12 [==============================] - ETA: 0s 

In [14]:
columns = ["batch_size", "epochs", "learning_rate", "loss_function", "images", "masks", "len_images", "len_masks", "channel", "image_size", "fold", "test_size", "val_size", "random_state", "path_dataset", "path_out", "data_augmentation", "filename_script", "time"]
data = [cfg["batch_size"], cfg["epochs"], cfg["learning_rate"], cfg["loss_function"], images_folder, masks_folder, len(list_images), len(list_masks), cfg["channel"], cfg["image_size"], cfg["fold"], cfg["test_size"], cfg["val_size"], cfg["random_state"], cfg["path_dataset"], cfg["path_out"], cfg["data_augmentation"], ipynbname.name(), time.strftime("%H:%M:%S", time.gmtime(end_time))]

dataframe_cfg = pandas.DataFrame(data, columns)
dataframe_cfg = dataframe_cfg.applymap(lambda x: str(x).replace(".", ",") if isinstance(x,float) else x)
dataframe_cfg.to_csv(os.path.join(path_model, "cfg.csv"), decimal=",", sep=";", na_rep=" ", header=False, quoting=csv.QUOTE_ALL)
dataframe_cfg

,0
batch_size,4
epochs,75
learning_rate,"0,001"
loss_function,dice
images,dataset/IMAGEM_ORIGINAL/CONVERTIDAS/RGB/512/OUT
masks,dataset/MASK/BITMAP/512/OUT
len_images,375
len_masks,375
channel,3
image_size,512
